In [ ]:
!pip install python-docx spacy pandas tqdm

In [ ]:
!pip install ollama

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

<Popen: returncode: None args: ['ollama', 'serve']>

In [ ]:
!ollama pull gpt-oss:20b

In [ ]:
import zipfile
import os

zip_file_path = "/content/TN 7.zip"
extract_dir = '/content'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"'{zip_file_path}' extracted successfully to '{extract_dir}'")

In [ ]:
import zipfile
import os

zip_file_path = "/content/TN_texts.zip"
extract_dir = "/content"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    for member in zip_ref.infolist():
        try:
            zip_ref.extract(member, extract_dir)
        except Exception as e:
            print(f"⚠️ Failed to extract {member.filename}: {e}")

print(f"✅ '{zip_file_path}' extracted successfully to '{extract_dir}'")


✅ '/content/TN_texts.zip' extracted successfully to '/content'


In [ ]:
import shutil
import os

folder_path = '/content/Cag'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
else:
    print("Folder does not exist.")


In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

<Popen: returncode: None args: ['ollama', 'serve']>

In [ ]:
import os
import re
import time
import json
import pandas as pd
from tqdm import tqdm
from docx import Document
import ollama

# ====== CONFIGURATION ======
MY_DOCS_FOLDER = r"/content/drive/MyDrive/TN"     # Folder with .docx files
RESULTS_FILE = r"/content/results.csv" # Output CSV
MODEL = "gpt-oss:20b"               # Ollama model

# Regex to stop reading text after PART II starts
PART_LINE_RE = re.compile(
    r"^\s*PART\s*[-\s]*([IVX]+)\b.*$",
    re.IGNORECASE,
)

# ---------- UTILITIES ----------
def extract_text_until_part2(doc_path):
    """Extract text from .docx until PART II heading. Return '' if file is invalid."""
    try:
        doc = Document(doc_path)
        lines = []
        for para in doc.paragraphs:
            text = para.text.strip()
            if PART_LINE_RE.match(text) and "II" in text.upper():
                break
            if text:
                lines.append(text)
        return "\n".join(lines)
    except Exception as e:
        print(f"⚠️ Skipping invalid file: {doc_path} ({e})")
        return ""


def clean_location(location):
    """Clean up location names to avoid full addresses."""
    if not location:
        return ""
    # If comma-separated, take last part (e.g., "Vadodara Highway Road, Halol" -> "Halol")
    if "," in location:
        return location.split(",")[-1].strip()
    # Remove words like 'Highway Road'
    location = re.sub(r"\b(highway|road|street|main|lane)\b.*", "", location, flags=re.IGNORECASE).strip()
    return location

def get_prompt():
    """Return strict JSON-only prompt for LLaMA3."""
    return (
"""You are an IR analyst. I will provide you the content of an IR file. Your task is to:

1. Identify the **State**, **Location**, and the most relevant **owning Department** of the office mentioned in the IR.
   - State = Name of the Indian state.
   - Location = Clean town/city/taluka name (avoid full addresses, keep only the place name like "Halol" or "Navsari").
   - Department = Only one overall department, always ending with the word "Department". Choose the most realistic and available department, last-owning administrative department.
2. Provide also a short **Explanation** in plain text about why you chose that department.
3. Return the answer **strictly in JSON** with the following keys:

{
  "state": "...",
  "location": "...",
  "department": "...",
  "explanation": "..."
}
⚠️ Rules:
- Do not add any extra text before or after the JSON.
- Do not wrap it in Markdown.
- Ensure values are plain text only.
- Make the department universal not specific to state and district or any other also no other content in that.
- Always return exactly these 4 fields with full forms not any abbrevations.
"""
    )
def analyze_ir_content(doc_text):
    """Send text to LLaMA3 and return parsed results (State, Location, Department, Explanation)."""
    if not doc_text.strip():
        return "", "", "", ""

    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": get_prompt()},
            {"role": "user", "content": doc_text},
        ]
    )

    content = response["message"]["content"].strip()

    try:
        parsed = json.loads(content)
        state = parsed.get("state", "").strip()
        location = clean_location(parsed.get("location", "").strip())
        department = parsed.get("department", "").strip()
        explanation = parsed.get("explanation", "").strip()
    except Exception as e:
        print("⚠️ JSON parse error:", e)
        print("Raw output:", content[:200])
        state, location, department, explanation = "", "", "", content

    return state, location, department, explanation


def main_process():
    rows = []

    for root, _, files in os.walk(MY_DOCS_FOLDER):
        folder_name = os.path.basename(root)
        for fname in tqdm(files, desc=f"Processing {folder_name}"):
            if not fname.lower().endswith(".docx") or fname.startswith("~$"):
                continue  # skip invalid/temp files

            fpath = os.path.join(root, fname)
            text = extract_text_until_part2(fpath)

            if not text.strip():  # skip empty/invalid files
                continue

            state, location, department, explanation = analyze_ir_content(text)

            rows.append({
                "Folder Name": folder_name,
                "Filename": fname,
                "State": state,
                "Location": location,
                "Department": department,
                "Explanation": explanation,
            })

            # Save progressively so we don’t lose progress
            df_existing = pd.DataFrame(rows)
            df_existing.to_csv(RESULTS_FILE, index=False)

    print(f"✅ Processing complete. Results saved in {RESULTS_FILE}")


if __name__ == "__main__":
    if not os.path.exists(RESULTS_FILE):
        pd.DataFrame(columns=[
            "Folder Name", "Filename", "State", "Location", "Department", "Explanation"
        ]).to_csv(RESULTS_FILE, index=False)

    main_process()

In [ ]:
import os
import re
import time
import json
import pandas as pd
from tqdm import tqdm
import ollama

# ====== CONFIGURATION ======
MY_TEXT_FOLDER = r"/content/TN_texts"  # Folder with .txt files
RESULTS_FILE = r"/content/results.csv"               # Output CSV
MODEL = "gpt-oss:20b"                                # Ollama model

# ---------- UTILITIES ----------
def load_text_file(file_path):
    """Safely load text from a .txt file."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        print(f"⚠️ Skipping invalid file: {file_path} ({e})")
        return ""

def clean_location(location):
    """Clean up location names to avoid full addresses."""
    if not location:
        return ""
    # If comma-separated, take last part (e.g., "Vadodara Highway Road, Halol" -> "Halol")
    if "," in location:
        return location.split(",")[-1].strip()
    # Remove words like 'Highway Road'
    location = re.sub(r"\b(highway|road|street|main|lane)\b.*", "", location, flags=re.IGNORECASE).strip()
    return location

def get_prompt():
    """Return strict JSON-only prompt for LLaMA3."""
    return (
"""You are an IR analyst. I will provide you the content of an IR file. Your task is to:

1. Identify the **State**, **Location**, and the most relevant **owning Department** of the office mentioned in the IR.
   - State = Name of the Indian state.
   - Location = Clean town/city/taluka name (avoid full addresses, keep only the place name like "Halol" or "Navsari").
   - Department = Only one overall department, always ending with the word "Department". Choose the most realistic and available department, last-owning administrative department.
2. Provide also a short **Explanation** in plain text about why you chose that department.
3. Return the answer **strictly in JSON** with the following keys:

{
  "state": "...",
  "location": "...",
  "department": "...",
  "explanation": "..."
}
⚠️ Rules:
- Do not add any extra text before or after the JSON.
- Do not wrap it in Markdown.
- Ensure values are plain text only.
- Make the department universal not specific to state and district or any other also no other content in that.
- Always return exactly these 4 fields with full forms not any abbrevations.
"""
    )

def analyze_ir_content(doc_text):
    """Send text to LLaMA3 and return parsed results (State, Location, Department, Explanation)."""
    if not doc_text.strip():
        return "", "", "", ""

    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": get_prompt()},
            {"role": "user", "content": doc_text},
        ]
    )

    content = response["message"]["content"].strip()

    try:
        parsed = json.loads(content)
        state = parsed.get("state", "").strip()
        location = clean_location(parsed.get("location", "").strip())
        department = parsed.get("department", "").strip()
        explanation = parsed.get("explanation", "").strip()
    except Exception as e:
        print("⚠️ JSON parse error:", e)
        print("Raw output:", content[:200])
        state, location, department, explanation = "", "", "", content

    return state, location, department, explanation

def main_process():
    rows = []

    files = [f for f in os.listdir(MY_TEXT_FOLDER) if f.lower().endswith(".txt") and not f.startswith("~$")]

    for fname in tqdm(files, desc="Processing text files"):
        fpath = os.path.join(MY_TEXT_FOLDER, fname)
        text = load_text_file(fpath)

        if not text.strip():  # skip empty/invalid files
            continue

        state, location, department, explanation = analyze_ir_content(text)

        rows.append({
            "Folder Name": os.path.basename(MY_TEXT_FOLDER),
            "Filename": fname,
            "State": state,
            "Location": location,
            "Department": department,
            "Explanation": explanation,
        })

        # Save progressively
        df_existing = pd.DataFrame(rows)
        df_existing.to_csv(RESULTS_FILE, index=False)

    print(f"✅ Processing complete. Results saved in {RESULTS_FILE}")

# ---------- RUN ----------
if __name__ == "__main__":
    if not os.path.exists(RESULTS_FILE):
        pd.DataFrame(columns=[
            "Folder Name", "Filename", "State", "Location", "Department", "Explanation"
        ]).to_csv(RESULTS_FILE, index=False)

    main_process()


Processing text files: 100%|██████████| 48/48 [10:07<00:00, 12.65s/it]

✅ Processing complete. Results saved in /content/results.csv
